In [1]:
import pandas as pd
import polars as pl

data = {
    "name": ["Tom", "Lisa", "John", "Vincent", "Mary", "Caroline"],
    "has_pet": ["Y", "N", "Y", "Y", "Y", "N"],
    "gender": ["M", "F", "M", "M", "F", "F"],
    "lucky_number": [19, 25, 36, 7, 2, 91],
}
df = pl.DataFrame(data)
print(df)

shape: (6, 4)
┌──────────┬─────────┬────────┬──────────────┐
│ name     ┆ has_pet ┆ gender ┆ lucky_number │
│ ---      ┆ ---     ┆ ---    ┆ ---          │
│ str      ┆ str     ┆ str    ┆ i64          │
╞══════════╪═════════╪════════╪══════════════╡
│ Tom      ┆ Y       ┆ M      ┆ 19           │
│ Lisa     ┆ N       ┆ F      ┆ 25           │
│ John     ┆ Y       ┆ M      ┆ 36           │
│ Vincent  ┆ Y       ┆ M      ┆ 7            │
│ Mary     ┆ Y       ┆ F      ┆ 2            │
│ Caroline ┆ N       ┆ F      ┆ 91           │
└──────────┴─────────┴────────┴──────────────┘


## Basic

In [2]:
print(df.group_by("has_pet").agg(pl.len(), pl.max("lucky_number")))

shape: (2, 3)
┌─────────┬─────┬──────────────┐
│ has_pet ┆ len ┆ lucky_number │
│ ---     ┆ --- ┆ ---          │
│ str     ┆ u32 ┆ i64          │
╞═════════╪═════╪══════════════╡
│ Y       ┆ 4   ┆ 36           │
│ N       ┆ 2   ┆ 91           │
└─────────┴─────┴──────────────┘


In [3]:
print(
    df.group_by(pl.col("name").str.len_bytes().alias("n_chars"))
    .agg(pl.len(), pl.col("name"))
    .sort("n_chars")
)

shape: (4, 3)
┌─────────┬─────┬──────────────────────────┐
│ n_chars ┆ len ┆ name                     │
│ ---     ┆ --- ┆ ---                      │
│ u32     ┆ u32 ┆ list[str]                │
╞═════════╪═════╪══════════════════════════╡
│ 3       ┆ 1   ┆ ["Tom"]                  │
│ 4       ┆ 3   ┆ ["Lisa", "John", "Mary"] │
│ 7       ┆ 1   ┆ ["Vincent"]              │
│ 8       ┆ 1   ┆ ["Caroline"]             │
└─────────┴─────┴──────────────────────────┘


## Conditional and filter

In [4]:
lt20 = pl.col("lucky_number").lt(20)

print(
    df.group_by("gender").agg(
        lt20.sum().alias("lt20"), lt20.not_().sum().alias("~lt20")
    )
)

shape: (2, 3)
┌────────┬──────┬───────┐
│ gender ┆ lt20 ┆ ~lt20 │
│ ---    ┆ ---  ┆ ---   │
│ str    ┆ u32  ┆ u32   │
╞════════╪══════╪═══════╡
│ M      ┆ 2    ┆ 1     │
│ F      ┆ 1    ┆ 2     │
└────────┴──────┴───────┘


In [5]:
print(
    df.group_by("gender").agg(
        pl.col("name").filter(pl.col("lucky_number").lt(20))
    )
)

shape: (2, 2)
┌────────┬────────────────────┐
│ gender ┆ name               │
│ ---    ┆ ---                │
│ str    ┆ list[str]          │
╞════════╪════════════════════╡
│ F      ┆ ["Mary"]           │
│ M      ┆ ["Tom", "Vincent"] │
└────────┴────────────────────┘


## Multiple columns or exprs

In [6]:
print(
    df.group_by("has_pet", "gender", maintain_order=True).agg(
        pl.col("name")
    )
)

shape: (3, 3)
┌─────────┬────────┬────────────────────────────┐
│ has_pet ┆ gender ┆ name                       │
│ ---     ┆ ---    ┆ ---                        │
│ str     ┆ str    ┆ list[str]                  │
╞═════════╪════════╪════════════════════════════╡
│ Y       ┆ M      ┆ ["Tom", "John", "Vincent"] │
│ N       ┆ F      ┆ ["Lisa", "Caroline"]       │
│ Y       ┆ F      ┆ ["Mary"]                   │
└─────────┴────────┴────────────────────────────┘


## Shortcut

In [7]:
print(df.group_by("has_pet", maintain_order=True).agg(pl.len()))

shape: (2, 2)
┌─────────┬─────┐
│ has_pet ┆ len │
│ ---     ┆ --- │
│ str     ┆ u32 │
╞═════════╪═════╡
│ Y       ┆ 4   │
│ N       ┆ 2   │
└─────────┴─────┘


In [8]:
print(df.group_by("has_pet", maintain_order=True).len())

shape: (2, 2)
┌─────────┬─────┐
│ has_pet ┆ len │
│ ---     ┆ --- │
│ str     ┆ u32 │
╞═════════╪═════╡
│ Y       ┆ 4   │
│ N       ┆ 2   │
└─────────┴─────┘


## Window function: pl.Expr.over()

In [9]:
print(df)

shape: (6, 4)
┌──────────┬─────────┬────────┬──────────────┐
│ name     ┆ has_pet ┆ gender ┆ lucky_number │
│ ---      ┆ ---     ┆ ---    ┆ ---          │
│ str      ┆ str     ┆ str    ┆ i64          │
╞══════════╪═════════╪════════╪══════════════╡
│ Tom      ┆ Y       ┆ M      ┆ 19           │
│ Lisa     ┆ N       ┆ F      ┆ 25           │
│ John     ┆ Y       ┆ M      ┆ 36           │
│ Vincent  ┆ Y       ┆ M      ┆ 7            │
│ Mary     ┆ Y       ┆ F      ┆ 2            │
│ Caroline ┆ N       ┆ F      ┆ 91           │
└──────────┴─────────┴────────┴──────────────┘


In [10]:
print(
    df.with_columns(
        pl.col("lucky_number")
        .rank("ordinal")
        .over("gender")
        .alias("rank_by_gender")
    )
)

shape: (6, 5)
┌──────────┬─────────┬────────┬──────────────┬────────────────┐
│ name     ┆ has_pet ┆ gender ┆ lucky_number ┆ rank_by_gender │
│ ---      ┆ ---     ┆ ---    ┆ ---          ┆ ---            │
│ str      ┆ str     ┆ str    ┆ i64          ┆ u32            │
╞══════════╪═════════╪════════╪══════════════╪════════════════╡
│ Tom      ┆ Y       ┆ M      ┆ 19           ┆ 2              │
│ Lisa     ┆ N       ┆ F      ┆ 25           ┆ 2              │
│ John     ┆ Y       ┆ M      ┆ 36           ┆ 3              │
│ Vincent  ┆ Y       ┆ M      ┆ 7            ┆ 1              │
│ Mary     ┆ Y       ┆ F      ┆ 2            ┆ 1              │
│ Caroline ┆ N       ┆ F      ┆ 91           ┆ 3              │
└──────────┴─────────┴────────┴──────────────┴────────────────┘


In [11]:
print(
    df.group_by("gender", maintain_order=True)
    .agg(
        pl.col("lucky_number").rank("ordinal").alias("rank_by_gender"),
        pl.all(),
    )
    .explode(pl.all().exclude("gender"))
    .select([*df.columns, "rank_by_gender"])
)

shape: (6, 5)
┌──────────┬─────────┬────────┬──────────────┬────────────────┐
│ name     ┆ has_pet ┆ gender ┆ lucky_number ┆ rank_by_gender │
│ ---      ┆ ---     ┆ ---    ┆ ---          ┆ ---            │
│ str      ┆ str     ┆ str    ┆ i64          ┆ u32            │
╞══════════╪═════════╪════════╪══════════════╪════════════════╡
│ Tom      ┆ Y       ┆ M      ┆ 19           ┆ 2              │
│ John     ┆ Y       ┆ M      ┆ 36           ┆ 3              │
│ Vincent  ┆ Y       ┆ M      ┆ 7            ┆ 1              │
│ Lisa     ┆ N       ┆ F      ┆ 25           ┆ 2              │
│ Mary     ┆ Y       ┆ F      ┆ 2            ┆ 1              │
│ Caroline ┆ N       ┆ F      ┆ 91           ┆ 3              │
└──────────┴─────────┴────────┴──────────────┴────────────────┘


In [12]:
df2 = df.with_columns(
    pl.Series("rank", [5, 6, 4, 1, 2, 3], dtype=pl.UInt32)
).select("name", "gender", "rank")
print(df2)

shape: (6, 3)
┌──────────┬────────┬──────┐
│ name     ┆ gender ┆ rank │
│ ---      ┆ ---    ┆ ---  │
│ str      ┆ str    ┆ u32  │
╞══════════╪════════╪══════╡
│ Tom      ┆ M      ┆ 5    │
│ Lisa     ┆ F      ┆ 6    │
│ John     ┆ M      ┆ 4    │
│ Vincent  ┆ M      ┆ 1    │
│ Mary     ┆ F      ┆ 2    │
│ Caroline ┆ F      ┆ 3    │
└──────────┴────────┴──────┘


In [13]:
print(
    df2.select(
        pl.all()
        .sort_by(pl.col("rank"))
        .over(pl.col("gender"), mapping_strategy="explode")
    )
)

shape: (6, 3)
┌──────────┬────────┬──────┐
│ name     ┆ gender ┆ rank │
│ ---      ┆ ---    ┆ ---  │
│ str      ┆ str    ┆ u32  │
╞══════════╪════════╪══════╡
│ Vincent  ┆ M      ┆ 1    │
│ John     ┆ M      ┆ 4    │
│ Tom      ┆ M      ┆ 5    │
│ Mary     ┆ F      ┆ 2    │
│ Caroline ┆ F      ┆ 3    │
│ Lisa     ┆ F      ┆ 6    │
└──────────┴────────┴──────┘


In [14]:
print(
    df2.with_columns(
        pl.col("rank")
        .sort()
        .over(pl.col("gender"), mapping_strategy="join")
    )
)

shape: (6, 3)
┌──────────┬────────┬───────────┐
│ name     ┆ gender ┆ rank      │
│ ---      ┆ ---    ┆ ---       │
│ str      ┆ str    ┆ list[u32] │
╞══════════╪════════╪═══════════╡
│ Tom      ┆ M      ┆ [1, 4, 5] │
│ Lisa     ┆ F      ┆ [2, 3, 6] │
│ John     ┆ M      ┆ [1, 4, 5] │
│ Vincent  ┆ M      ┆ [1, 4, 5] │
│ Mary     ┆ F      ┆ [2, 3, 6] │
│ Caroline ┆ F      ┆ [2, 3, 6] │
└──────────┴────────┴───────────┘


In [15]:
print(
    df2.select(
        pl.all()
        .sort_by(pl.col("rank"))
        # for each gender, get the first row after sorting by "rank"
        .head(1)
        .over(pl.col("gender"), mapping_strategy="explode")
    )
)

shape: (2, 3)
┌─────────┬────────┬──────┐
│ name    ┆ gender ┆ rank │
│ ---     ┆ ---    ┆ ---  │
│ str     ┆ str    ┆ u32  │
╞═════════╪════════╪══════╡
│ Vincent ┆ M      ┆ 1    │
│ Mary    ┆ F      ┆ 2    │
└─────────┴────────┴──────┘


In [16]:
print(
    df2.select(
        pl.all()
        .sort_by(pl.col("rank"))
        .over(pl.col("gender"), mapping_strategy="explode")
        .head(1)
    )
)

shape: (1, 3)
┌─────────┬────────┬──────┐
│ name    ┆ gender ┆ rank │
│ ---     ┆ ---    ┆ ---  │
│ str     ┆ str    ┆ u32  │
╞═════════╪════════╪══════╡
│ Vincent ┆ M      ┆ 1    │
└─────────┴────────┴──────┘


## codepanda

In [17]:
df_pd = pd.DataFrame(data)
df_pd

,name,has_pet,gender,lucky_number
0,Tom,Y,M,19
1,Lisa,N,F,25
2,John,Y,M,36
3,Vincent,Y,M,7
4,Mary,Y,F,2
5,Caroline,N,F,91


In [18]:
print(
    df_pd.groupby("has_pet").agg(
        len=("has_pet", "size"),
        lucky_number=("lucky_number", "max"),
    )
)

         len  lucky_number
has_pet                   
N          2            91
Y          4            36


In [19]:
print(
    df_pd.assign(n_chars=df_pd["name"].str.len())
    .groupby("n_chars")
    .agg(len=("name", "size"), name=("name", list))
)

         len                name
n_chars                         
3          1               [Tom]
4          3  [Lisa, John, Mary]
7          1           [Vincent]
8          1          [Caroline]


In [20]:
print(
    df_pd.assign(
        rank_by_gender=lambda df_: df_.groupby("gender")
        .lucky_number.transform(lambda s_: s_.rank())
        .astype(int)
    )
)

       name has_pet gender  lucky_number  rank_by_gender
0       Tom       Y      M            19               2
1      Lisa       N      F            25               2
2      John       Y      M            36               3
3   Vincent       Y      M             7               1
4      Mary       Y      F             2               1
5  Caroline       N      F            91               3


## Remarks

### Remark2

In [21]:
lt20 = pl.col("lucky_number").lt(20)
ge20 = pl.col("lucky_number").ge(20)


print(
    df.group_by("gender").agg(
        lt20.sum().alias("lt20"), ge20.sum().alias("ge20")
    )
)

shape: (2, 3)
┌────────┬──────┬──────┐
│ gender ┆ lt20 ┆ ge20 │
│ ---    ┆ ---  ┆ ---  │
│ str    ┆ u32  ┆ u32  │
╞════════╪══════╪══════╡
│ F      ┆ 1    ┆ 2    │
│ M      ┆ 2    ┆ 1    │
└────────┴──────┴──────┘
